# Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
transactions = pd.read_pickle(r'data\03_int\int_transactions.pkl')
offers = pd.read_pickle(r'data/03_int/int_offers.pkl')
offers_per_cust = pd.read_pickle(r'data\04_fct\fct_offers_per_customer.pkl')
transactions_per_cust = pd.read_pickle(r'data\04_fct\fct_transactions_per_cust.pkl')
profile_feat = pd.read_pickle(r'data\03_int\int_profile_feat.pkl')

# Match Transactions and Offers

In [3]:
# Inner join the transactions and offers dataframes on 'customer_id'
merged = pd.merge(transactions, offers, on='customer_id', how='inner')

# Sort by 'transaction' and 'offer_received'
merged.sort_values(['transaction', 'offer_received'], inplace=True)

# Filter transactions that occurred during the offer period
filtered_transactions = merged[(merged['transaction'] >= merged['offer_received']) & 
                               (merged['transaction'] <= merged['offer_completed'])]

# Count the number of transactions and the total transaction amount for each customer and offer
agg_matched_transactions = filtered_transactions.groupby(['customer_id', 'offer_id']).agg(
    total_transactions=('transaction_amount', 'count')
    ,total_transaction_amount=('transaction_amount', 'sum')
    # ,transaction_list=('transaction', list) # uncomment to use as a check
    # ,amount_list=('transaction_amount', list) # uncomment to use as a check
).reset_index()

# Check num customers in the dataframe. Should be less than 17000
num_customers = agg_matched_transactions['customer_id'].nunique()
print(f'There were {num_customers} customers who made transactions during the offer period.')

# Check the maximum number of transactions for a single customer. should be less than 36
max_transactions = agg_matched_transactions['total_transactions'].max()
print(f'The maximum number of transactions for a single customer was {max_transactions}.')

agg_matched_transactions.head()


There were 12774 customers who made transactions during the offer period.
The maximum number of transactions for a single customer was 22.


,customer_id,offer_id,total_transactions,total_transaction_amount
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,10.27
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,8.57
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,14.11
3,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,22.05
4,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,1,11.93


# Calculate Transactions and Amount per Offer

In [4]:
# We want to left join the offers dataframe with the aggregated transactions dataframe
# This allows us to see
    # 1. Offers that were sent out and completed
    # 2. Offers that were sent out but not completed
df_matched = pd.merge(offers, agg_matched_transactions, on=['customer_id', 'offer_id'], how='left')

# Fill NaN values with 0
df_matched['total_transactions'] = df_matched['total_transactions'].fillna(0)
df_matched['total_transaction_amount'] = df_matched['total_transaction_amount'].fillna(0)

# Inner join the profile features to the dataframe
# -> We want to inner join so that we can filter out customers who did not receive any offers
df_matched = pd.merge(profile_feat, df_matched, on='customer_id', how='inner')

# Save data
df_matched.to_pickle(r'data\04_fct\fct_matched_offers.pkl')
df_matched.to_csv(r'data\04_fct\fct_matched_offers.csv', index=False)

# Calculate the number of unique customers. Should be 16,994
num_customers = df_matched['customer_id'].nunique()
print(f'There are {num_customers} unique customers in the dataset.')

print(df_matched.shape)
df_matched.head()

There are 16994 unique customers in the dataset.
(63288, 27)


,customer_id,age,income,days_as_member,gender_F,gender_M,gender_O,gender_Unknown,offer_id,offer_received,...,email,mobile,social,web,is_bogo,is_discount,is_informational,expiration,total_transactions,total_transaction_amount
0,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,529,0,0,0,1,2906b810c7d4411798c6938adc9daaa5,168.0,...,1,1,0,1,0,1,0,336.0,0.0,0.00
1,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,529,0,0,0,1,0b1e1539f2cc45b7b9fa7c272da2e1d7,336.0,...,1,0,0,1,0,1,0,576.0,0.0,0.00
2,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,529,0,0,0,1,fafdcd668e3743c1bb461111dcafc2a4,408.0,...,1,1,1,1,0,1,0,648.0,5.0,10.17
3,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,529,0,0,0,1,2298d6c36e964ae4a3e7e9706d1fb8c2,504.0,...,1,1,1,1,0,1,0,672.0,3.0,7.54
4,0610b486422d4921ae7d2bf64640c50b,55.0,112000.0,376,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,408.0,...,1,1,0,1,1,0,0,576.0,1.0,23.22


# Identify Customers that made transactions, but did not receive offers

In [5]:
#Create a data frame of customer_ids that did not receive an offer
no_offer = profile_feat[~profile_feat['customer_id'].isin(df_matched['customer_id'])]

# Merge trans_per_cust on customer_id
no_offer = pd.merge(no_offer, transactions_per_cust, on='customer_id', how='left')

# Calculate the number of unique customers. Should be 6
num_customers = no_offer['customer_id'].nunique()
print(f'There are {num_customers} unique customers in the dataset.')

# Export data
no_offer.to_csv(r'data\04_fct\fct_customers_no_offer.csv', index=False)
no_offer.to_pickle(r'data\04_fct\fct_customers_no_offer.pkl')

no_offer.head()

There are 6 unique customers in the dataset.


,customer_id,age,income,days_as_member,gender_F,gender_M,gender_O,gender_Unknown,total_transactions,total_transaction_amount
0,c6e579c6821c41d1a7a6a9cf936e91bb,72.0,35000.0,285,1,0,0,0,4,6.95
1,da7a7c0dcfcb41a8acc7864a53cf60fb,NaN,NaN,359,0,0,0,1,1,0.35
2,eb540099db834cf59001f83a4561aef3,66.0,34000.0,300,1,0,0,0,4,26.01
3,3a4874d8f0ef42b9a1b72294902afea9,55.0,88000.0,709,1,0,0,0,3,67.62
4,ae8111e7e8cd4b60a8d35c42c1110555,54.0,72000.0,566,1,0,0,0,5,86.43


# Aggregate offers, transactions and matched offer_transactions on profiles

In [6]:
### Join transactions to customer profiles ###
# Left join transactions per customer on profile_feat
agg_obt = pd.merge(profile_feat, transactions_per_cust, on='customer_id', how='left')


### Join matched offers to customer profiles ###
# Calculate the total matched transaction amount and total number of matched transactions for each customer
total_matched_transactions = agg_matched_transactions.groupby('customer_id').agg(
                                                                total_offer_transaction_amount=('total_transaction_amount', 'sum'),
                                                                total_offer_transactions=('total_transactions', 'sum')
                                                            ).reset_index()

# Left join total_matched_transactions on profile_feat
agg_obt = pd.merge(agg_obt, total_matched_transactions, on='customer_id', how='left')

# # Fill NaN values with 0
agg_obt['total_offer_transaction_amount'] = agg_obt['total_offer_transaction_amount'].fillna(0)
agg_obt['total_offer_transactions'] = agg_obt['total_offer_transactions'].fillna(0)
agg_obt['total_transactions'] = agg_obt['total_transactions'].fillna(0)
agg_obt['total_transaction_amount'] = agg_obt['total_transaction_amount'].fillna(0)

### Join offers_per_cust to customer profiles ###
# Left join offers_per_cust on profile_feat
agg_obt = pd.merge(agg_obt, offers_per_cust, on='customer_id', how='left')

# Check to make sure the number of unique customers is correct
num_cust = agg_obt['customer_id'].nunique()
print(f'There are {num_cust} unique customers in the dataset.')

# Export data
agg_obt.to_pickle(r'data\04_fct\fct_agg_obt.pkl')
agg_obt.to_csv(r'data\04_fct\fct_agg_obt.csv', index=False)

agg_obt.head()

There are 17000 unique customers in the dataset.


,customer_id,age,income,days_as_member,gender_F,gender_M,gender_O,gender_Unknown,total_transactions,total_transaction_amount,...,offers_viewed,informational_offers,offers_completed,offers_viewed_before_completion,viewed_before_completion_reward,viewed_after_completion_reward,eligible_rewards,percent_of_offers_viewed,percent_of_offers_completed,percent_of_offers_viewed_before_completion
0,68be06ca386d4c31939f3a4f0e3dd783,NaN,NaN,529,0,0,0,1,9.0,20.40,...,4.0,0.0,2.0,2.0,5.0,0.0,12.0,1.000000,0.5,1.000000
1,0610b486422d4921ae7d2bf64640c50b,55.0,112000.0,376,1,0,0,0,3.0,77.01,...,0.0,1.0,1.0,0.0,0.0,5.0,5.0,0.000000,1.0,0.000000
2,38fe809add3b4fcf9315a9694bb96ff5,NaN,NaN,14,0,0,0,1,6.0,14.30,...,2.0,1.0,0.0,0.0,0.0,0.0,5.0,1.000000,0.0,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,75.0,100000.0,443,1,0,0,0,7.0,159.27,...,4.0,1.0,3.0,2.0,15.0,5.0,20.0,1.000000,1.0,0.666667
4,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,356,0,0,0,1,3.0,4.65,...,2.0,2.0,0.0,0.0,0.0,0.0,5.0,0.666667,0.0,NaN
